### Transformaciones de datos

In [1]:
# Import libreries
import pandas as pd
from pdfquery import PDFQuery
import os
import pdfplumber
import numpy as np
import sqlalchemy as sql
# import pyodbc
import time

Database Connection

In [2]:
server = 'ASUSTUF\SQL22'; database = 'AdventureWorksDW2019'
try:
    engine = sql.create_engine(f"mssql+pyodbc://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")
    print('Conexión exitosa')
except sql.Error as e:
    print(f'Error al conectar a la base de datos: {str(e)}')

current_time = time.strftime("%H:%M:%S", time.localtime())

Conexión exitosa


Variables necesarias

In [3]:
# Tabla Destino
table_name = "tasasInteres_TMP"
esquema = "dbo"

#Diccionario de meses
month_dict = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

In [4]:
# Ruta directorio
root_dir = os.getcwd()
print(root_dir)

c:\Users\rfigu\Documents\Python Scripts\WebScrapp_pdf


In [5]:
# Definimos las Columnas de la pagina 1 del pdf Resumen de Tasas
colPag1= ['Banco', 'MinMax', 'TC_ML_Nominal', 'TC_ML_Efectiva', 'TC_ME_Nominal', 'TC_ME_Efectiva', 
          'Micro_ML_Nominal', 'Micro_ML_Efectiva','Micro_ME_Nominal','Micro_ME_Efectiva', 
          'VivAFD_ML_Nominal','VivAFD_ML_Efectiva','VivAFD_ME_Nominal','VivAFD_ME_Efectiva'
         ,'VivPro_ML_Nominal','VivPro_ML_Efectiva','VivPro_ME_Nominal','VivPro_ME_Efectiva']

# Definimos las Columnas de la pagina 1 del pdf Resumen de Tasas
colPag2= ['Banco', 'MinMax', 'PCom_ML_Nominal', 'PCom_ML_Efectiva', 'PCom_ME_Nominal', 'PCom_ME_Efectiva', 
          'PDes_ML_Nominal','PDes_ML_Efectiva','PDes_ME_Nominal','PDes_ME_Efectiva',
          'PPer_ML_Nominal','PPer_ML_Efectiva','PPer_ME_Nominal','PPer_ME_Efectiva'
         ,'Sobre_ML_Nominal','Sobre_ML_Efectiva','Sobre_ME_Nominal','Sobre_ME_Efectiva']

# Definimos las Columnas de la pagina 1 del pdf Resumen de Tasas
colPag3= ['Banco', 'MinMax', 'AH_ML_Nominal', 'AH_ML_Efectiva', 'AH_ME_Nominal', 'AH_ME_Efectiva', 
          'Cda180_ML_Nominal','Cda180_ML_Efectiva','Cda180_ME_Nominal','Cda180_ME_Efectiva',
          'Cda365_ML_Nominal','Cda365_ML_Efectiva','Cda365_ME_Nominal','Cda365_ME_Efectiva'
         ,'CDA_ML_Nominal','CDA_ML_Efectiva','CDA_ME_Nominal','CDA_ME_Efectiva']

In [8]:
df_union.head()

,Banco,MinMax,TipoTasa,value
0,Banco Nacional de Fomento,Mínima,TC_ML_Nominal,"1 4,00"
1,None,Máxima,TC_ML_Nominal,"1 4,00"
2,Banco de la Nación Argentina,Mínima,TC_ML_Nominal,-
3,None,Máxima,TC_ML_Nominal,-
4,Banco GNB Paraguay S.A.,Mínima,TC_ML_Nominal,"1 6,07"


In [12]:
df_union["TipoCredito"] = df_union["TipoTasa"].str[:2]
df_union.head()

,Banco,MinMax,TipoTasa,value,TipoCredito
0,Banco Nacional de Fomento,Mínima,TC_ML_Nominal,"1 4,00",TC
1,None,Máxima,TC_ML_Nominal,"1 4,00",TC
2,Banco de la Nación Argentina,Mínima,TC_ML_Nominal,-,TC
3,None,Máxima,TC_ML_Nominal,-,TC
4,Banco GNB Paraguay S.A.,Mínima,TC_ML_Nominal,"1 6,07",TC


In [13]:
df_union["MlME"] = df_union["TipoTasa"].str[3:5]
df_union.head(2)

,Banco,MinMax,TipoTasa,value,TipoCredito,MlME
0,Banco Nacional de Fomento,Mínima,TC_ML_Nominal,"1 4,00",TC,ML
1,None,Máxima,TC_ML_Nominal,"1 4,00",TC,ML


In [14]:
df_union["NomiEfect"] = df_union["TipoTasa"].str[6:]
df_union.head(2)

,Banco,MinMax,TipoTasa,value,TipoCredito,MlME,NomiEfect
0,Banco Nacional de Fomento,Mínima,TC_ML_Nominal,"1 4,00",TC,ML,Nominal
1,None,Máxima,TC_ML_Nominal,"1 4,00",TC,ML,Nominal


In [ ]:
current_time = time.strftime("%H:%M:%S", time.localtime())
lstArch = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            # Crear cadena ruta archivo
            file_paths = os.path.join(dirpath, filename)
            lstArch.append(filename)
            print(filename)
        
            with pdfplumber.open(file_paths) as pdf:
    # -----------------Seccion de Extracción de datos
                first_page = pdf.pages[0]          #-- Pag 1
                second_page = pdf.pages[1]         #-- Pag 2         
                third_page = pdf.pages[2]          #-- Pag 3
                #  
                # Obtener la Fecha Cierre del reporte de Tasas
                textoFecha = first_page.extract_text()
                ini = str(textoFecha).find("MES:")
                fin = str(textoFecha).find("\n")
                cadena = textoFecha[ini+5:fin]
                
                mes, anio = cadena.split('/')
                            # print(f"Fecha : {mes}, {anio}")  
                # Use a lambda function to replace the month name with its corresponding number
                converted_month = month_dict.get(mes.lower(), mes)

                fch = f"01-{converted_month}-{anio}"
                print(f"Fecha Cierre del Archivo: {fch}")
                #---------------------------------
                datatable1 = first_page.extract_tables()
                datatable2 = second_page.extract_tables()
                datatable3 = third_page.extract_tables()
                #
                df_pag1 = pd.DataFrame(datatable1)
                df_pag2 = pd.DataFrame(second_page.extract_tables())
                df_pag3 = pd.DataFrame(third_page.extract_tables())
                # Creamos un arrays de listas
                vector1 = df_pag1.iloc[0].values
                vector2 = df_pag2.iloc[0].values
                vector3 = df_pag3.iloc[0].values
                # Transformamos el vector en lista
                lstvector1 = list(vector1)
                df_tasas1 = pd.DataFrame(lstvector1, columns=colPag1)

                lstvector2 = list(vector2)
                df_tasas2 = pd.DataFrame(lstvector2, columns=colPag2)

                lstvector3 = list(vector3)
                df_tasas3 = pd.DataFrame(lstvector3, columns=colPag3)

                # Eliminar primeras filas pag 1
                df_tasas1.drop(df_tasas1.index[0:4], inplace=True)
                # Eliminar primeras filas pag 2
                df_tasas2.drop(df_tasas2.index[0:4], inplace=True)
                # Eliminar primeras filas pag 3
                df_tasas3.drop(df_tasas3.index[0:4], inplace=True)
                # Reindexar el indice
                df_tasas1.reset_index(drop=True, inplace=True)
                df_tasas2.reset_index(drop=True, inplace=True)
                df_tasas3.reset_index(drop=True, inplace=True)

                # hasta aqui optenemos los datos de las tres hojas por separadado

                #----------------  Seccion para desagrupar --------------

                # Seleccionar columnas que comienzan con 'M'
                col_MEyML = [col for col in df_tasas1.columns if '_M' in col]
                # Desagrupar DataFrame
                df_unpivot1 = pd.melt(df_tasas1, id_vars=['Banco', 'MinMax'], value_vars= col_MEyML, var_name='TipoTasa')

                # Ver DataFrame desagrupado
                # df_unpivot1.head()        #------------------------ check

                # Seleccionar columnas que comienzan con 'M'
                col_MEyML = [col for col in df_tasas2.columns if '_M' in col]
                # Desagrupar DataFrame
                df_unpivot2 = pd.melt(df_tasas2, id_vars=['Banco', 'MinMax'], value_vars= col_MEyML, var_name='TipoTasa')

                # Ver DataFrame desagrupado
                # df_unpivot2.head()        #------------------------- check

                # Seleccionar columnas que comienzan con 'M'
                col_MEyML = [col for col in df_tasas3.columns if '_M' in col]
                # Desagrupar DataFrame
                df_unpivot3 = pd.melt(df_tasas3, id_vars=['Banco', 'MinMax'], value_vars= col_MEyML, var_name='TipoTasa')

                # Ver DataFrame desagrupado
                # df_unpivot3.head()        #------------------------ check

                df_union = pd.concat([df_unpivot1, df_unpivot2, df_unpivot3], ignore_index=True)
               

                # df_union.tail()

                # Reemplazamos la coma con el punto
                df_union.replace(',', '.', regex=True, inplace=True)
                # eliminarmos espacios en blanco
                df_union['value'] = df_union['value'].replace(' ', '', regex=True)

                # Eliminar filas que contienen el carácter "-"
                df_f1 = df_union[~df_union['value'].str.contains('-')].copy()

                # Reiniciar Indices
                df_f1 = df_f1.reset_index(drop=True)

                # Rellenar nombre bancos
                df_f1.fillna(method='ffill', inplace=True)

                # Agregando FechaCierre
                df_f1['FechaCierre']=fch

                # Agregando FechaCierre
                df_f1['FechaProceso']=current_time

                # Cambiar tipo de datos
                df_f1['value'] = df_f1['value'].astype(float)
                df_f1['FechaProceso'] = pd.to_datetime(df_f1['FechaProceso'], errors='coerce')
                df_f1['FechaCierre'] = pd.to_datetime(df_f1['FechaCierre'], errors='coerce', format='%d/%m/%Y')
                
                # Creando columnas para dimension
                df_f1["TipoCredito"] = df_f1["TipoTasa"].str[:2]
                df_f1["MlME"] = df_f1["TipoTasa"].str[3:5]
                df_f1["NomiEfect"] = df_f1["TipoTasa"].str[6:]
                # Agregamos el nombre del Archivo
                df_f1['FileName']=filename
                        
                print(f"Filas y columnas a insertar: {df_union.shape}")
                df_f1.to_sql(table_name, engine, schema=esquema, if_exists='append', index=False)
                print(f"Filas insertadas: {df_f1.shape} ")

                        

In [ ]:
df_f1.dtypes

Bloque de Conexion a Base, con pyodbc

In [ ]:
import pyodbc

server = 'ASUSTUF\SQL22'; database = '@database'

try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                                f'SERVER={server};'
                                f'DATABASE={database};'
                                f'Trusted_Connection=yes;')
    print('Conexión exitosa')
except pyodbc.Error as e:
    print(f'Error al conectar a la base de datos: {str(e)}')

bloque conexion con SQL sqlalchemy

In [ ]:
with pdfplumber.open('descargas\BANCOS%200122.pdf') as pdf:
    # -----------------Seccion de Extracción de datos
    first_page = pdf.pages[0]          #-- Pag 1
    second_page = pdf.pages[1]         #-- Pag 2         
    third_page = pdf.pages[2]          #-- Pag 3
 
    textoFecha = first_page.extract_text()
    
    ini = str(textoFecha).find("MES:")
    fin = str(textoFecha).find("\n")
    cadena = textoFecha[ini+5:fin]
    print(cadena)
    
    mes, anio = cadena.split('/')
    print(f"Fecha : {mes}, {anio}")  
    # Use a lambda function to replace the month name with its corresponding number
    converted_month = month_dict.get(mes.lower(), mes)

    fch = f"01-{converted_month}-{anio}"

    print(f"Fecha Cierre: {fch}")